In [29]:
import pandas as pd 
import numpy as np
import csv
import pickle

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [30]:
full_input = pd.read_csv('merged1.csv')
full_input = full_input.iloc[:,1:]

In [31]:
print(full_input)

      tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0              0.202351          -0.021717          -0.111851   
1              0.282814          -0.016686          -0.110004   
2              0.272529          -0.020954          -0.114472   
3              0.276408          -0.024315          -0.101067   
4              0.306400          -0.013045          -0.064768   
...                 ...                ...                ...   
2055           0.275866          -0.015977          -0.107413   
2056           0.276938          -0.010382          -0.104780   
2057           0.275988          -0.010583          -0.107595   
2058           0.293323          -0.008314          -0.086588   
2059          -0.503823          -0.594207           0.264804   

      tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  \
0            -0.332095         -0.114114         -0.309439         -0.382582   
1            -0.994285         -0.973997         -0.988410 

In [32]:
act_input = full_input.iloc[:,:561]
act_true = full_input.iloc[:,561]

print(act_input.shape)
print(act_input)
print(act_true)

(2060, 561)
      tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0              0.202351          -0.021717          -0.111851   
1              0.282814          -0.016686          -0.110004   
2              0.272529          -0.020954          -0.114472   
3              0.276408          -0.024315          -0.101067   
4              0.306400          -0.013045          -0.064768   
...                 ...                ...                ...   
2055           0.275866          -0.015977          -0.107413   
2056           0.276938          -0.010382          -0.104780   
2057           0.275988          -0.010583          -0.107595   
2058           0.293323          -0.008314          -0.086588   
2059          -0.503823          -0.594207           0.264804   

      tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  \
0            -0.332095         -0.114114         -0.309439         -0.382582   
1            -0.994285         -0.973997       

In [33]:
loc_input = full_input.iloc[:,562:-6]   
loc_true = full_input.iloc[:,-6:]   

print(loc_input.shape)
print(loc_input)
print(loc_true)

(2060, 520)
      WAP079  WAP168  WAP117  WAP221  WAP482  WAP113  WAP345  WAP313  WAP176  \
0          0       0       0       0       0       0       0       0       0   
1          0       0      10       0       0       0       0       0       0   
2          0       0       0       0       0       0       0       0       0   
3          0       0       0       0       0       0       0       0       0   
4          0       0       0       0       0       0       0       0       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
2055       0       0       0       0       0       0      33       0       0   
2056       0       0       0       0       0       0       0       0       0   
2057       0       0       0       0       0       0       0       0       0   
2058       0       0       0       0       0       0       0       0       0   
2059       0       0       0       0       0       0       0       0       0   

      WAP068  ...  WAP251  

In [34]:
activity_model = pickle.load(open('activity.sav', 'rb'))

loc_models = []
y_cols = ["LONGITUDE","LATITUDE","FLOOR","BUILDINGID","SPACEID","RELATIVEPOSITION"]

for i in range(len(y_cols)):
    loc_models.append(pickle.load(open(y_cols[i]+str('.sav'), 'rb')))

In [35]:
def read_user_id(n):
    print("Enter User Id between 0-"+str(n)+":")
    user_id = int(input())
    while user_id<0 or user_id>=200:
        print("Enter User Id between 0-"+str(n)+":")
        user_id = int(input())
    return user_id

def predict(user_id):
    pred = [user_id]
    activity_input = (np.array(act_input.iloc[user_id,:])).reshape(1,-1)
    pred.append(activity_model.predict(activity_input))
    
    localisation_input = (np.array(loc_input.iloc[user_id,:])).reshape(1,-1)
    for i in range(len(y_cols)):
        pred.append(loc_models[i].predict(localisation_input))
    
    print("User {} is {} on Floor ID: {} ,Building ID: {}; Location Coordinates : {},{}\n".format(pred[0], pred[1],pred[4],pred[5],pred[2],pred[3])) 
    print("G.Truth: {} on Floor ID: {} ,Building ID: {}; Location Coordinates : {},{}\n".format( act_true[user_id],loc_true.iloc[user_id,2],loc_true.iloc[user_id,3],loc_true.iloc[user_id,0],loc_true.iloc[user_id,1])) 
    #print(pred)
    return pred


In [36]:
n = len(act_true)
predict(read_user_id(n))

Enter User Id between 0-2060:
10
User 10 is ['STANDING'] on Floor ID: [3] ,Building ID: [1]; Location Coordinates : [158.0254235],[174.27151113]

G.Truth: STANDING on Floor ID: 3 ,Building ID: 1; Location Coordinates : 154.67525492864843,175.03278402984142



[10,
 array(['STANDING'], dtype=object),
 array([158.0254235]),
 array([174.27151113]),
 array([3]),
 array([1]),
 array([104]),
 array([2])]

In [39]:
import sklearn.metrics as sm
y_hat = activity_model.predict(act_input)

In [40]:
print(sm.confusion_matrix(act_true,y_hat))
print(sm.classification_report(act_true, y_hat))

[[406   0   0   0   0   0]
 [  0 361  16   0   0   0]
 [  0  14 340   0   0   0]
 [  0   0   0 366   2   1]
 [  0   0   0   3 275   6]
 [  0   0   0   1   4 265]]
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       406
           SITTING       0.96      0.96      0.96       377
          STANDING       0.96      0.96      0.96       354
           WALKING       0.99      0.99      0.99       369
WALKING_DOWNSTAIRS       0.98      0.97      0.97       284
  WALKING_UPSTAIRS       0.97      0.98      0.98       270

          accuracy                           0.98      2060
         macro avg       0.98      0.98      0.98      2060
      weighted avg       0.98      0.98      0.98      2060



In [42]:
for i in range(2):    
    yhat = loc_models[i].predict(loc_input)    
    print("====================",y_cols[i],"===============")
    print("MAE: ",sm.mean_absolute_error(loc_true[y_cols[i]], yhat))
    print("Score: ",sm.r2_score(loc_true[y_cols[i]], yhat))
    

==================== LONGITUDE ===============
MAE:  3.075958213595428
Score:  0.9958361859725791
==================== LATITUDE ===============
MAE:  2.4903527600658553
Score:  0.991959640750192


In [44]:
for i in range(2,6):
    yhat = loc_models[i].predict(loc_input)
    print("====================",y_cols[i],"===============")  
    print(sm.classification_report(loc_true[y_cols[i]], yhat))
    print(" ")    

==================== FLOOR ===============
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       451
           1       1.00      1.00      1.00       547
           2       0.99      1.00      1.00       440
           3       0.99      1.00      0.99       511
           4       1.00      0.99      1.00       111

    accuracy                           0.99      2060
   macro avg       1.00      0.99      0.99      2060
weighted avg       0.99      0.99      0.99      2060

 
==================== BUILDINGID ===============
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       589
           1       0.99      1.00      0.99       523
           2       1.00      0.99      1.00       948

    accuracy                           1.00      2060
   macro avg       1.00      1.00      1.00      2060
weighted avg       1.00      1.00      1.00      2060

 
==================== SPACEID ======